# Backend Logic 
## R Hackathon

In [ ]:
#devtools::install_github("condwanaland/words")
#devtools::install_git("https://github.com/condwanaland/words.git")
#https://github.com/condwanaland/words.git
install.packages("words")

### Read in packages

In [ ]:
#Packages----
library(words)
library(stringr)
library(dplyr)


In [ ]:
##WordList----
n_length <- 5 #input variable

word_list <- words::words %>% 
  filter(word_length == n_length) %>% 
  select(word)

In [ ]:
#Guess----
##UI Guess----
guess <- sample(word_list$word,1) # used for testing - to be rm in final version
#guess <- "pikis" #input variable

##Server word----
target <- sample(word_list$word,1) # input variable
#target <- "conin" # used for testing - to be rm in final version

#create lists
guess_val <- strsplit(guess, "")[[1]]
target_val <- strsplit(target, "")[[1]]

### Functions

#### Exit Game

In [ ]:
##Guess length---
guess_length <- function(target, guess) {
  if(nchar(guess) != nchar(target)){
    stop("Error - word not the correct length")
  }
}

In [ ]:
#Word In list----
guess_in_list <-  function(target, guess) {
  if(!(guess %in% word_list$word)){
    stop("Error - not a real word")
  }
}

#### Continue Game

In [ ]:
##Continue game functions----
word_checker <- function(target_val, guess_val) {
  output <- character(n_length)
  target_remaining <- target_val
  guess_remaining <- character(n_length)
  
  #target frequencies
  target_freq <- data.frame(target_val) %>% 
    group_by(target_val) %>% 
    summarize(target_freq = n()) %>% #count the frequency of each letter in the target
    ungroup()
  #frequencies
  freq <- data.frame(guess_val) %>% 
    mutate(position = 1:n_length) %>% #get position of letter in word
    group_by(guess_val) %>% 
    mutate(guess_freq = n()) %>% #count the frequency of each letter in the guess
    ungroup() %>% 
    left_join(target_freq, by = c("guess_val" = "target_val")) %>% #join frequencies of guess and target together
    mutate(target_freq = ifelse(is.na(target_freq) == TRUE, 0, target_freq)) # replace NA with 0 where target letter does not appear in guess
  #minimum position flag
  freq <- freq %>% group_by(guess_val) %>% 
    mutate(counter = row_number(guess_val)) %>% #get position of specific letter
    ungroup()
  
  
  for (i in 1:n_length){
    
    if(guess_val[i] == target_val[i]) {
      output[i] <- "Correct"
      target_remaining[i] <- NA
      
    } else { #needs to only highlight 1 
      #guess
      guess_remaining[i] <-  guess_val[i]
      output[i] <- "Not"
    }
  }
  #Add counts to this for loop  
  for (i in 1:n_length){
    if((guess_val[i] != target_val[i]) & (guess_remaining[i] %in% target_remaining)){
      #check counts
      if(freq[i,3] <= freq[i,4]){
        #if target counts = guess counts then 
        output[i] <-"In"
        #if guess count != target count 
      }else if(freq[i,3] > freq[i,4] & freq[i,4] >= freq[i,5]){ #and in min position then 
        #only flag first case
        output[i] <- "In"
        #not in
      }  }
  }
  output <- data.frame(output)
  return(output)
  
}

In [ ]:
#Function outputs
guess_length(target, guess)
guess_in_list(target, guess)

#Continue game functions ----
output <- word_checker(target_val, guess_val)
output <- data.frame(guess_val, output)
print(output)
